# Работа с Excel

Материалы:
* Макрушин С.В. Лекция 7: Работа с Excel
* https://docs.xlwings.org/en/stable/quickstart.html
* https://nbviewer.jupyter.org/github/pybokeh/jupyter_notebooks/blob/master/xlwings/Excel_Formatting.ipynb#search_text


## Задачи для совместного разбора

1. На листе "Рецептура" файла `себестоимостьА_в1.xlsx` для области "Пшеничный хлеб" рассчитать себестоимость всех видов продукции.

2. Результаты расчетов 1.1 сохранить в отдельном столбце области "Пшеничный хлеб"

3. Приблизить форматирование столбца, добавленного в задаче 2 к оформлению всей области.

4. Выполнить 3 с помощью "протягиваемых" формул.

## Лабораторная работа 7.1

In [2]:
import pandas as pd

1. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample.csv` (__ЛР5__) в виде `pd.DataFrame`. Обратите внимание на корректное считывание столбца(ов) с индексами. Оставьте в таблице с рецептами следующие столбцы: `id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`

In [2]:
reviews = pd.read_csv('data/reviews_sample.csv', index_col=0)
recipes = pd.read_csv('data/recipes_sample.csv', index_col=0, usecols=['id', 'name', 'minutes', 'submitted', 'description', 'n_ingredients'])

In [3]:
print(reviews.head())

           user_id  recipe_id        date  rating  \
370476       21752      57993  2003-05-01       5   
624300      431813     142201  2007-09-16       5   
187037      400708     252013  2008-01-10       4   
706134  2001852463     404716  2017-12-11       5   
312179       95810     129396  2008-03-14       5   

                                                   review  
370476  Last week whole sides of frozen salmon fillet ...  
624300  So simple and so tasty!  I used a yellow capsi...  
187037  Very nice breakfast HH, easy to make and yummy...  
706134  These are a favorite for the holidays and so e...  
312179  Excellent soup!  The tomato flavor is just gre...  


In [4]:
print(recipes.head())

                                             id  minutes   submitted  \
name                                                                   
george s at the cove  black bean soup     44123       90  2002-10-25   
healthy for them  yogurt popsicles        67664       10  2003-07-26   
i can t believe it s spinach              38798       30  2002-08-29   
italian  gut busters                      35173       45  2002-07-27   
love is in the air  beef fondue   sauces  84797       25  2004-02-23   

                                                                                description  \
name                                                                                          
george s at the cove  black bean soup     an original recipe created by chef scott meska...   
healthy for them  yogurt popsicles        my children and their friends ask for my homem...   
i can t believe it s spinach                        these were so go, it surprised even me.   
italian  gut busters

2. Случайным образом выберите 5% строк из каждой таблицы и сохраните две таблицы на разные листы в один файл `recipes.xlsx`. Дайте листам названия "Рецепты" и "Отзывы", соответствующие содержанию таблиц. 

In [5]:
reviews_sample = reviews.sample(frac=0.05)
recipes_sample = recipes.sample(frac=0.05)


with pd.ExcelWriter('recipes.xlsx') as writer:
    recipes_sample.to_excel(writer, sheet_name='Рецепты')
    reviews_sample.to_excel(writer, sheet_name='Отзывы')


3. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_assign`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи присваивания массива значений диапазону ячеек.

In [3]:
import xlwings as xw

In [6]:
book = xw.Book('recipes.xlsx')

sheet = book.sheets['Рецепты']

recipes_df = sheet.range('A1').options(pd.DataFrame, header=1, index=False, expand='table').value


recipes_df['seconds_assign'] = recipes_df['minutes'] * 60
header = sheet.range('G1')
header.value = 'seconds_assign'

# Присваиваем массив значений диапазону ячеек на листе
sheet.range('G2').options(transpose=True).value = recipes_df['seconds_assign'].values


book.save()
book.close()

4. Используя `xlwings`, добавьте на лист `Рецепты` столбец `seconds_formula`, показывающий время выполнения рецепта в секундах. Выполните задание при помощи формул Excel.

In [7]:
book = xw.Book('recipes.xlsx')

sheet = book.sheets['Рецепты']

sheet.range('H1').value = 'seconds_formula'

# Вычисляем формулу для каждой ячейки в столбце "seconds_formula"
last_row = sheet.range('A1').current_region.last_cell.row
sheet.range('H2:H{}'.format(last_row)).formula = '=C2*60'


book.save()
book.close()

5. Сделайте названия всех добавленных столбцов полужирными и выровняйте по центру ячейки.

In [9]:
book = xw.Book('recipes.xlsx')
sheet = book.sheets['Рецепты']

# Выделяем ячейки, содержащие заголовки столбцов, и применяем форматирование
headers = sheet.range('A1:H1')
headers.api.Font.Bold = True
headers.api.HorizontalAlignment = xw.constants.HAlign.xlHAlignCenter


book.save()
book.close()

6. Раскрасьте ячейки столбца `minutes` в соответствии со следующим правилом: если рецепт выполняется быстрее 5 минут, то цвет - зеленый; от 5 до 10 минут - жёлтый; и больше 10 - красный.

In [10]:
book = xw.Book('recipes.xlsx')
sheet = book.sheets['Рецепты']

# Определяем цвета ячеек
green = (0, 255, 0)
yellow = (255, 255, 0)
red = (255, 0, 0)

# Применяем форматирование к столбцу "minutes"
last_row = sheet.range('A1').current_region.last_cell.row
for i in range(2, last_row + 1):
    minutes = sheet.range('C{}'.format(i)).value
    if minutes < 5:
        color = green
    elif minutes < 10:
        color = yellow
    else:
        color = red
    sheet.range('C{}'.format(i)).color = color


book.save()
book.close()

7. Добавьте на лист `Рецепты`  столбец `n_reviews`, содержащий кол-во отзывов для этого рецепта. Выполните задание при помощи формул Excel.

In [11]:
book = xw.Book('recipes.xlsx')
sheet = book.sheets['Рецепты']

# Добавляем столбец "n_reviews" на лист
sheet.range('I1').value = 'n_reviews'

# Вычисляем формулу для каждой ячейки в столбце "n_reviews"
last_row = sheet.range('A1').current_region.last_cell.row
sheet.range('I2:I{}'.format(last_row)).formula = '=COUNTIF(Отзывы!C:C, B2)'


book.save()
book.close()

## Лабораторная работа 7.2

8. Напишите функцию `validate()`, которая проверяет соответствие всех строк из листа `Отзывы` следующим правилам:
    * Рейтинг - это число от 0 до 5 включительно
    * Соответствующий рецепт имеется на листе `Рецепты`
    
В случае несоответствия этим правилам, выделите строку красным цветом

In [19]:
def validate():
    wb = xw.Book('recipes.xlsx')
    recipes_sheet = wb.sheets['Рецепты']
    reviews_sheet = wb.sheets['Отзывы']
    
    # Определяем цвет для выделения строк
    red = (255, 0, 0)
    
    # Получаем диапазон ячеек на листе "Отзывы"
    last_row = reviews_sheet.range('A1').current_region.last_cell.row
    range_str = 'A2:F{}'.format(last_row)
    range = reviews_sheet.range(range_str)
    
    # Получаем список id рецептов из листа Рецепты
    recipe_ids = recipes_sheet.range('B2:B{}'.format(recipes_sheet.range('B1').current_region.last_cell.row)).value
    
    # Проверяем соответствие правилам для каждой строки в диапазоне
    for row in range.rows:
        rating = row[4].value
        if rating is None or not (0 <= rating <= 5):
            row.color = red
            continue
        
        recipe_id = row[2].value
        if recipe_id is None or recipe_id not in recipe_ids:
            row.color = red
            continue
    
    wb.save()
    wb.close()

validate()


9. В файле `recipes_model.csv` находится модель данных предметной области "рецепты". При помощи пакета `csv` считайте эти данные. При помощи пакета `xlwings` запишите данные на лист `Модель` книги `recipes_model.xlsx`, начиная с ячейки `A2`, не используя циклы. Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука. 

In [4]:
import csv

In [39]:
with open('data/recipes_model.csv', encoding='utf-8') as f:
    reader = csv.reader(f, delimiter='\t')
    data = list(reader)

# Создаем новую книгу и получаем лист "Модель"
wb = xw.Book()
wb.sheets[0].name = 'Модель'
sheet = wb.sheets['Модель']

# Записываем данные на лист
sheet.range('A2').value = data

wb.save('recipes_model.xlsx')
wb.close()


![scr_1](scr1.png)

10. При помощи пакета `xlwings` добавьте в столбец J формулу для описания столбца на языке SQL. Формула должна реализовывать следующую логику:

    1\. в начале строки идут значения из столбцов В и C (значение столбца С приведено к верхнему регистру), разделенные пробелом
    
    2\. далее идут слова на основе столбца "Ключ"
        2.1 если в столбце "Ключ" указано значение "PK", то дальше через пробел идет ключевое слово "PRIMARY KEY"
        2.2 если в столбце "Ключ" указано значение "FK", то дальше через пробел идет ключевое слово "REFERENCES", затем значения столбцов H и I в формате "название_таблицы(название_столбца)"
        
    3\. если в столбце "Обязательно к заполнению" указано значение "Y" и в столбце "Ключ" указано не "PK", то дальше через пробел идет ключевое слово "NOT NULL".

Заполните этой формулой необходимое количество строк, используя "протягивание". Количество строк для протягивания определите на основе данных.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [71]:
wb = xw.Book('recipes_model.xlsx')
sheet = wb.sheets['Модель']

# определяем количество строк для протягивания формулы
num_rows = sheet.range('A1').current_region.last_cell.row

# добавляем формулу в столбец J
for i in range(2, num_rows + 1):
    b_val = sheet.range(f'B{i}').value
    c_val = sheet.range(f'C{i}').value.upper()
    key_val = sheet.range(f'G{i}').value
    h_val = sheet.range(f'H{i}').value
    i_val = sheet.range(f'I{i}').value
    j_formula = f'{b_val} {c_val}'
    
    if key_val == 'PK':
        j_formula += ' PRIMARY KEY'
    elif key_val == 'FK':
        j_formula += f' REFERENCES {h_val}({i_val})'
    
    req_val = sheet.range(f'D{i}').value
    if req_val == 'Y' and key_val != 'PK':
        j_formula += ' NOT NULL'
    
    sheet.range(f'J{i}').formula = j_formula

wb.save()
wb.close()


![scr2](scr2.png)

11. При помощи пакета `xlwings` измените стилизацию листа `Модель`.
* для заголовков добавьте заливку цвета `00ccff`
* примените автоподбор ширины столбца;
* сделайте шрифт заголовков полужирным;
* добавьте таблице автофильтр.

Сделайте скриншот текущего состояния листа и прикрепите в ячейку ноутбука.

In [5]:
wb = xw.Book('recipes_model.xlsx')
ws = wb.sheets['Модель']

# Заливка цветом для заголовков
header_range = ws.range('A1:J1')
header_range.color = (0, 204, 255)

# Автоподбор ширины столбца
ws.autofit()

# Жирный шрифт для заголовков
header_range.api.Font.Bold = True

# Добавление автофильтра
table_range = ws.range('A1:J1')  
table_range.api.AutoFilter() 


wb.save()
wb.close()

![scr3](scr3.png)

12. Посчитайте количество атрибутов для каждой из сущностей. Создайте лист `Статистика` и запишите в него результат группировки, начиная с ячейки "А1". Визуализируйте полученный результат при помощи столбчатой диаграммы. Сохраните полученную визуализацию на лист `Статистика`, начиная с ячейки "E2".  Сделайте скриншот листа `Статистика` и прикрепите в ячейку ноутбука.

* Вы можете воспользоваться методами для визуализации, которые поставляются вместе с объектами `pandas` (см. https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot) 